In [ ]:
# Let's refactor the code for optimization
from datetime import datetime, timedelta
import random
import pandas as pd

def get_meteorological_season(date):
    month = date.month
    if month >= 3 and month <= 5:
        return "Spring"
    elif month >= 6 and month <= 8:
        return "Summer"
    elif month >= 9 and month <= 11:
        return "Fall"
    else:
        return "Winter"

# Create the date range
date_list = [datetime(2022, 12, 15) + timedelta(days=i) for i in range(17)]  # 17 days

# Simulating products_df with 125 products for the sake of example
# You'll actually read this from a CSV file
products_df = pd.DataFrame({
    'product_id': range(1, 126),
    'product_season': ['Winter']*25 + ['Spring']*25 + ['Summer']*25 + ['Fall']*25 + ['General']*25
})

# Initialize variables
inventory_data = []
inventory_key = 0
units_balance_dict = {}

# Loop through dates and products
for date in date_list:
    for index, row in products_df.iterrows():
        product_id = row['product_id']
        season = row['product_season']
        
        if product_id not in units_balance_dict:
            units_balance_dict[product_id] = 0
            
        # 50% chance to generate UnitsIn
        if random.random() < 0.5:
            units_in = random.randint(10, 20)
            units_balance_dict[product_id] += units_in
            inventory_key += 1
            inventory_data.append({
                'InventoryMovementKey': inventory_key,
                'MovementDate': date,
                'ProductN': product_id,
                'UnitsIn': units_in,
                'UnitsOut': 0,
                'UnitsBalance': units_balance_dict[product_id]
            })
            
        # Chance to generate 0-5 rows for UnitsOut based on season
        current_meteorological_season = get_meteorological_season(date)
        if season == "General":
            num_rows_out = random.randint(1, 7)
        elif season == current_meteorological_season:
            num_rows_out = random.randint(2, 5)
        else:
            num_rows_out = random.randint(1, 3)
        
        for _ in range(num_rows_out):
            if units_balance_dict[product_id] == 0:
                break
            if season == "General":
                units_out = random.randint(1, 7)
            elif season == current_meteorological_season:
                units_out = random.randint(1, 4)
            else:
                units_out = random.randint(1, 2)
                
            if units_balance_dict[product_id] - units_out < 0:
                units_out = units_balance_dict[product_id]
            units_balance_dict[product_id] -= units_out
            inventory_key += 1
            inventory_data.append({
                'InventoryMovementKey': inventory_key,
                'MovementDate': date,
                'ProductN': product_id,
                'UnitsIn': 0,
                'UnitsOut': units_out,
                'UnitsBalance': units_balance_dict[product_id]
            })

# Create DataFrame from list of dictionaries
inventory_df = pd.DataFrame(inventory_data)
inventory_df.head()
